In [1]:
import pandas as pd

df_eacl_kannada_train = pd.read_csv("../data/kannada data/kannada_offensive_train.csv",encoding="utf-8",sep="\t",usecols=[0,1])
df_eacl_kannada_dev = pd.read_csv("../data/kannada data/kannada_offensive_dev.csv",encoding="utf-8",sep="\t",usecols=[0,1])
df_eacl_kannada_test = pd.read_csv("../data/kannada data/kannada_offensive_test_with_labels.csv",encoding="utf-8",sep="\t",usecols=[0,1])

df_eacl_kannada_train = df_eacl_kannada_train.dropna()
df_eacl_kannada_dev = df_eacl_kannada_dev.dropna()
df_eacl_kannada_test = df_eacl_kannada_test.dropna()

print(df_eacl_kannada_train.info())

eacl_kannada_train_sentences = list(df_eacl_kannada_train.values[:,0])
eacl_kannada_train_labels = list(df_eacl_kannada_train.values[:,1])

eacl_kannada_dev_sentences = list(df_eacl_kannada_dev.values[:,0])
eacl_kannada_dev_labels = list(df_eacl_kannada_dev.values[:,1])

eacl_kannada_test_sentences = list(df_eacl_kannada_test.values[:,0])
eacl_kannada_test_labels = list(df_eacl_kannada_test.values[:,1])

x_train = eacl_kannada_train_sentences
y_train = eacl_kannada_train_labels

x_dev = eacl_kannada_dev_sentences
y_dev = eacl_kannada_dev_labels

x_test = eacl_kannada_test_sentences
y_test = eacl_kannada_test_labels

print(len(x_train),len(x_dev),len(x_test)," total = ",len(x_train)+len(x_dev)+len(x_test))

print(set(eacl_kannada_train_labels))
print(set(eacl_kannada_dev_labels))
print(set(eacl_kannada_test_labels))

total_sentences = []
total_sentences.extend(x_train)
total_sentences.extend(x_dev)
total_sentences.extend(x_test)

total_labels = []
total_labels.extend(eacl_kannada_train_labels)
total_labels.extend(eacl_kannada_dev_labels)
total_labels.extend(eacl_kannada_test_labels)

from collections import Counter
Counter(total_labels)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6216 entries, 0 to 6215
Data columns (total 2 columns):
 #   Column                                                                                  Non-Null Count  Dtype 
---  ------                                                                                  --------------  ----- 
 0   Tik tok alli jagala madtidralla adra baggenu ondu video madi anna super agi ugididdira  6216 non-null   object
 1   Not_offensive                                                                           6216 non-null   object
dtypes: object(2)
memory usage: 145.7+ KB
None
6216 776 777  total =  7769
{'Offensive_Untargetede', 'not-Kannada', 'Not_offensive', 'Offensive_Targeted_Insult_Other', 'Offensive_Targeted_Insult_Individual', 'Offensive_Targeted_Insult_Group'}
{'not-Kannada', 'Offensive_Targeted_Insult_Other', 'Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede'}
{'not-Kannada', 'Of

Counter({'not-Kannada': 1898,
         'Not_offensive': 4395,
         'Offensive_Targeted_Insult_Group': 417,
         'Offensive_Targeted_Insult_Other': 153,
         'Offensive_Targeted_Insult_Individual': 628,
         'Offensive_Untargetede': 278})

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(total_sentences)
print(X.shape)

(7769, 12860)


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
cosine_sim = cosine_similarity(X,X)

In [4]:
sum_rows_cosine_sim = np.sum(cosine_sim,axis=1)

In [5]:
dict_cosine_sim = {}
for i in range(len(y_train)+len(y_dev),sum_rows_cosine_sim.shape[0]):
    dict_cosine_sim[i] = sum_rows_cosine_sim[i]

In [6]:
sorted_dict = dict(sorted(dict_cosine_sim.items(), key=lambda item: item[1],reverse=True))

In [7]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = take(200, sorted_dict.items())

In [8]:
dict_key_dissmilar={}

for key,value in n_items:
    similar_values = cosine_sim[key]
    count_dissimilar_labels = 0
    count_similar_labels=0
    key_label = total_labels[key]
    if value > 0:
        for i in range(len(similar_values)):
            if cosine_sim[key][i]>0:
                if key_label == total_labels[i]:
                    count_similar_labels+=1
                else:
                    count_dissimilar_labels+=1
        diff = count_dissimilar_labels-count_similar_labels
        dict_key_dissmilar[key] = diff


In [9]:
sorted_key_dict = dict(sorted(dict_key_dissmilar.items(), key=lambda item: item[1],reverse=True))
print(sorted_key_dict)

{7479: 1806, 7524: 895, 7197: 714, 7003: 637, 7433: 613, 7357: 545, 7156: 492, 7717: 491, 7157: 482, 7641: 482, 7277: 439, 7023: 427, 7312: 413, 6994: 406, 7072: 406, 7103: 406, 7110: 406, 7179: 406, 7620: 405, 7616: 397, 7229: 394, 7763: 393, 7475: 383, 7230: 382, 7242: 382, 7188: 382, 7725: 370, 7582: 353, 7443: 352, 7280: 341, 7591: 341, 7429: 338, 7177: 334, 7273: 334, 7278: 334, 7351: 334, 7580: 334, 7196: 319, 7613: 318, 7313: 313, 7090: 282, 7726: 274, 7465: 263, 7419: 256, 7381: 244, 7662: 244, 7151: 243, 7374: 237, 7311: 234, 7371: 234, 7087: 233, 7514: 233, 7590: 233, 7669: 233, 7508: 229, 7399: 228, 7275: 220, 7416: 220, 7041: 211, 7306: 207, 7133: 204, 7307: 204, 7766: 197, 7331: 196, 7618: 188, 7645: 185, 7123: 185, 7497: 173, 7245: 171, 7218: 114, 7251: 105, 7745: 91, 7742: 90, 7302: 89, 7660: 53, 7361: 52, 7267: 25, 7265: 25, 7448: 21, 7058: 12, 7030: -5, 7125: -61, 7405: -62, 7585: -80, 7694: -84, 7289: -99, 7614: -111, 7001: -120, 7392: -123, 7195: -129, 7509: -129, 70

In [10]:
n_key_items = take(50, sorted_key_dict.items())

In [11]:
from collections import Counter
dict_key_dissmilar_label_count={}
for key,value in n_key_items:
    dissimilar_labels=[]
    similar_values = cosine_sim[key]
    key_label = total_labels[key]
    for i in range(len(similar_values)):
        if cosine_sim[key][i]>0:
            if key_label != total_labels[i]:
                dissimilar_labels.append(total_labels[i])
    counted = Counter(dissimilar_labels)
    most_common_counts = counted.most_common()
    dict_key_dissmilar_label_count[key] = most_common_counts

In [12]:
for key,value in dict_key_dissmilar_label_count.items():
    print(key,value)

7479 [('Not_offensive', 1065), ('not-Kannada', 594), ('Offensive_Targeted_Insult_Group', 67), ('Offensive_Targeted_Insult_Individual', 62), ('Offensive_Untargetede', 45)]
7524 [('Not_offensive', 835), ('Offensive_Targeted_Insult_Individual', 69), ('Offensive_Untargetede', 56), ('Offensive_Targeted_Insult_Other', 24), ('not-Kannada', 5)]
7197 [('Not_offensive', 591), ('Offensive_Targeted_Insult_Group', 55), ('Offensive_Targeted_Insult_Individual', 39), ('Offensive_Untargetede', 38), ('not-Kannada', 4)]
7003 [('Not_offensive', 347), ('not-Kannada', 233), ('Offensive_Targeted_Insult_Group', 46), ('Offensive_Targeted_Insult_Individual', 25), ('Offensive_Targeted_Insult_Other', 16)]
7433 [('Not_offensive', 514), ('not-Kannada', 51), ('Offensive_Targeted_Insult_Group', 36), ('Offensive_Untargetede', 28), ('Offensive_Targeted_Insult_Other', 9)]
7357 [('Not_offensive', 1041), ('Offensive_Targeted_Insult_Group', 38), ('Offensive_Targeted_Insult_Individual', 36), ('Offensive_Untargetede', 27), (

In [13]:
qa_data = []
qa_labels=[]
for key,value in n_key_items:
    if value>0:
        qa_data.append(total_sentences[key])
        qa_labels.append(total_labels[key])

In [14]:
qa_df = pd.DataFrame(zip(qa_data,qa_labels),columns=['qa_data','qa_labels'])
qa_df.to_csv('kannada_eacl_qa.tsv',sep='\t',index=None)